<a href="https://colab.research.google.com/github/niksche/Diploma-thesis-Designing-communications-satellite-/blob/master/antenna-reflector-dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>